# Building a neural network

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

In [2]:
# get the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device being used is {device}")

Device being used is cpu


In [3]:
# define the neural network class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device) # simulate a grayscale image
logits = model(X) # pass it through the model to get an output of 10 digits
# the logit output are not yet probabilities since some values are negative 
pred_probab = nn.Softmax(dim=1)(logits) # convert the logit to into probabilities that sum up to 1 
y_pred = pred_probab.argmax(1) # get the index of the highest prob for each input
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
# flatten a multi-dimensional tensor into a 1D vector
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20) # take 784 input values and returns 20 outputs 
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
# show the effect of the ReLU
print(f"Before ReLU:\n {hidden1}\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: \n {hidden1}\n")

Before ReLU:
 tensor([[ 0.3506, -0.4134,  0.4005, -0.2202, -0.4817,  0.0489,  0.0561,  0.0200,
          0.3957, -0.3950, -0.3222, -0.0595,  0.0458,  0.1764,  0.0499,  0.2379,
         -0.1046,  0.5136, -0.4195,  0.0355],
        [ 0.4518, -0.2673,  0.2230, -0.2378,  0.1521,  0.3072,  0.1363,  0.0833,
          0.0994, -0.6954, -0.4337, -0.0410,  0.1866,  0.0362,  0.2019,  0.1840,
         -0.0467,  0.0222, -0.7236,  0.3454],
        [ 0.4580, -0.0163,  0.2171,  0.0498, -0.3014,  0.0583, -0.1977,  0.0971,
          0.0754, -0.2773, -0.6065,  0.0822,  0.3726,  0.2909, -0.4652,  0.4671,
          0.0424,  0.3450, -0.5463,  0.4504]], grad_fn=<AddmmBackward0>)

After ReLU: 
 tensor([[0.3506, 0.0000, 0.4005, 0.0000, 0.0000, 0.0489, 0.0561, 0.0200, 0.3957,
         0.0000, 0.0000, 0.0000, 0.0458, 0.1764, 0.0499, 0.2379, 0.0000, 0.5136,
         0.0000, 0.0355],
        [0.4518, 0.0000, 0.2230, 0.0000, 0.1521, 0.3072, 0.1363, 0.0833, 0.0994,
         0.0000, 0.0000, 0.0000, 0.1866, 0.0362, 0.

In [79]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [81]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [82]:
print(f"Model structure: {model}")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0045, -0.0055,  0.0247,  ...,  0.0254,  0.0356,  0.0051],
        [ 0.0069,  0.0282,  0.0252,  ..., -0.0237,  0.0086,  0.0068]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0084, -0.0111], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0021,  0.0397,  0.0178,  ...,  0.0427, -0.0148, -0.0300],
        [ 0.0302,  0.0362,  0.0339,  ...,  0.0066,  0.0349,  0.0416]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: